# YELP API Data on businesses: City of Los Angeles CA
--------------------------------------------

## ETL PROCES REPORT
--------------------------------------------
### E: PULLING DATA FROM YELP'S API:

* Step 1: Get an API KEY  from yelp
* Investigate what kind of data to retrieve. In our case: General business profiles in the City of LA California
* The API returns only 50 results max per call
* Perform a test pull to look at the result and it's json structure to design the data frame

### T: PREPROCESSING THE JSON RESPONSE FROM THE API:
--------------------------------------------

* Declare/Initialize the variables/columns that will hold the downloaded data
* We build a dataset of 1000 resutls by making multiple calls dynamically and saving data into a dataframe on the fly
* Data processing consisted in extracting data from the json response and formatting it to create tabular data for the df
* The Transforming process included string manipulation regex and python scripting.

### L: LOADING TO SQL AND MONGODB:
--------------------------------------------
* This was the most straightforward step accomplished with no issues
* We simplify this proces by using sqlite instead of mysql or postgress
* We also saved csv files of the data before we loaded it into the database systems.


In [5]:
import requests
import json
import pandas as pd
from credentials import APIKey
import time

In [6]:
MY_API_KEY = APIKey
HEADERS = {'Authorization': 'bearer %s' % MY_API_KEY}


In [7]:
base_url = "https://api.yelp.com/v3/businesses/search"

In [8]:
search_location = 'los angeles ca united states'

In [9]:
PARAMETERS = {'location': search_location,
                  'limit': 50,
                  'radius': 40000,
                 }

In [10]:
response = requests.get(url = base_url,
                        params = PARAMETERS,
                        headers = HEADERS)

### Check if response was succesful: response code 200 🕺🏽

In [15]:
print(response)

<Response [200]>


In [ ]:
response.text

In [16]:
responseDict = {}

In [17]:
response_url = response.url
print(f"{response_url}")
print(f"type(response_url): {type(response_url)}")

https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000
type(response_url): <class 'str'>


In [20]:
json_response = response.json()

In [25]:
# inspect the json file
json_response['businesses'][0]

{'id': 'TkFEKhsCixPWlShULKvMdQ',
 'alias': 'bottega-louie-los-angeles',
 'name': 'Bottega Louie',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/rAImnKvUNcNY8i6qEDWrZA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/bottega-louie-los-angeles?adjust_creative=rCpoZ5I2XhS-AiKMOroCkg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=rCpoZ5I2XhS-AiKMOroCkg',
 'review_count': 16243,
 'categories': [{'alias': 'italian', 'title': 'Italian'},
  {'alias': 'bakeries', 'title': 'Bakeries'},
  {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}],
 'rating': 4.0,
 'coordinates': {'latitude': 34.0469300995766, 'longitude': -118.256601457672},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '700 S Grand Ave',
  'address2': None,
  'address3': '',
  'city': 'Los Angeles',
  'zip_code': '90017',
  'country': 'US',
  'state': 'CA',
  'display_address': ['700 S Grand Ave', 'Los Angeles, CA 90017']},
 'phone': '+12138021470',
 'display_phon

In [26]:
type(response.text)

str

In [27]:
json_response.keys()

dict_keys(['businesses', 'total', 'region'])

In [32]:
# Save the body of the first call to API(FIRST 50 RESULTS)
responseDict[response_url] = response.text

In [36]:
# Save a copy of the json file as a json in local machine
loop1_text = 'Resources/loop1_response.txt'

In [37]:
with open(loop1_text, 'w') as text:
    text.writelines(response.text)
    

In [ ]:
# responseDict

In [38]:
print(type(json_response['total']))
print(len(json_response['businesses']))
print(f"atl_json_response.keys(): {json_response.keys()}")
print(f"atl_response['total']: {json_response['total']}")

<class 'int'>
50
atl_json_response.keys(): dict_keys(['businesses', 'total', 'region'])
atl_response['total']: 11000


In [38]:
biz_count = json_response['total']
biz_count

11000

In [40]:
# test joining business categories  from the response object into one string
', '.join([val['title'] for val in json_response['businesses'][0]['categories']])

'Italian, Bakeries, Breakfast & Brunch'

In [41]:
# The API will only privide 1000 results max. Set the variable to a max of 1000 to use in following API calls
if biz_count > 1000:
    biz_count = 1000

In [42]:
biz_count

1000

In [43]:
# variables to use in second loop. bins to dynamically divide the calls into chunks
print(biz_count)
bins = []
temp_urls = []
response_url

1000


'https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000'

In [44]:
# In case the response has fewer than 1000 results, this will create the appropriate amount of urls
for i in range(50, biz_count, 50):
    bins.append(i)
    
print(bins)

[50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950]


In [45]:
# loop to build the amount of urls according to the amount of results from the response object
for i in range(len(bins)):
    temp_urls.append(response_url + "&offset=" + f"{str(bins[i])}")

In [47]:
# inspect the first and last url to test the chunks
print(temp_urls[0],'\n', temp_urls[-1])

https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=50 
 https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=950


### Initialize an empty data frame to dump resutls of API CALL INTO:

In [48]:
biz_df = pd.DataFrame()

In [49]:
biz_df['id'] = ''
biz_df['alias'] = ''
biz_df['name'] = ''
biz_df['categories'] = ''
biz_df['review_count'] = ''
biz_df['rating'] = ''
biz_df['lat'] = ''
biz_df['long'] = ''
# biz_df['transactions'] = ''
biz_df['address'] = ''
biz_df['city'] = ''
biz_df['zip_code'] = ''
biz_df['country'] = ''
biz_df['state'] = ''
biz_df['phone'] = ''


In [50]:
# Inspect the last result
json_response['businesses'][-1]

{'id': 'sYn3SNQP-j2t2XSwjlCbRg',
 'alias': 'montys-good-burger-los-angeles',
 'name': "Monty's Good Burger",
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/qoVy9kU9SLr8dsJNuIweWA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/montys-good-burger-los-angeles?adjust_creative=rCpoZ5I2XhS-AiKMOroCkg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=rCpoZ5I2XhS-AiKMOroCkg',
 'review_count': 1201,
 'categories': [{'alias': 'burgers', 'title': 'Burgers'},
  {'alias': 'vegan', 'title': 'Vegan'}],
 'rating': 4.5,
 'coordinates': {'latitude': 34.06469, 'longitude': -118.30876},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '516 S Western Ave',
  'address2': '',
  'address3': None,
  'city': 'Los Angeles',
  'zip_code': '90020',
  'country': 'US',
  'state': 'CA',
  'display_address': ['516 S Western Ave', 'Los Angeles, CA 90020']},
 'phone': '+12139150257',
 'display_phone': '(213) 915-0257',
 'distance': 1206.434807

In [51]:
# Inspect the urls to be used in dynamic api calls from result 50 onwards. Each one will return 50 results
for i in range(len(temp_urls)):
    print(temp_urls[i])

https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=50
https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=100
https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=150
https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=200
https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=250
https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=300
https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=350
https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=40000&offset=400
https://api.yelp.com/v3/businesses/search?location=los+angeles+ca+united+states&limit=50&radius=4

In [53]:
#Outside variable that will count results and serve as an index as df is built
# THIS WILL keep track of the results in both loops: first 50 results and the rest:
# in this case it is 1000
index_ = 0


In [ ]:
# responseDict[response_url] = response.text
#Loop #1: Grab data from first API CALL performed earlier
#And injecti it into the empty dataframe
try:
    for j in range(len(json_response['businesses'])):
        print("=======================================================")
        print(f"Injecting DataFrame with first set of 50 results")
        print(f"biz index: {index_}: {json_response['businesses'][j]['name']}")
        print("=======================================================")
        
#         detail = temp_response_json['businesses'][j]
        
        biz_df.set_value(index_, "id", json_response['businesses'][j]['id'])
        biz_df.set_value(index_, "alias", json_response['businesses'][j]['alias'])

        biz_df.set_value(index_, "name", json_response['businesses'][j]['name'])
        biz_df.set_value(index_, "categories", ', '.join([val['title'] for val in json_response['businesses'][j]['categories']]) )        

        biz_df.set_value(index_, "review_count", json_response['businesses'][j]['review_count'])
        biz_df.set_value(index_, "rating", json_response['businesses'][j]['rating'])

        biz_df.set_value(index_, "lat", json_response['businesses'][j]['coordinates']['latitude'])
        biz_df.set_value(index_, "long", json_response['businesses'][j]['coordinates']['longitude'])

        biz_df.set_value(index_, "address", ' '.join(json_response['businesses'][j]['location']['display_address']))
        biz_df.set_value(index_, "city", json_response['businesses'][j]['location']['city'])

        biz_df.set_value(index_, "zip_code", json_response['businesses'][j]['location']['zip_code'])
        biz_df.set_value(index_, "country", json_response['businesses'][j]['location']['country'])

        biz_df.set_value(index_, "state", json_response['businesses'][j]['location']['state'])
        biz_df.set_value(index_, "phone", json_response['businesses'][j]['phone'])

        index_ += 1
except:
    print("Something went WRONG 😫😓")            
              

In [56]:
# Inspect the resulting dataframe after the first loop
biz_df.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,zip_code,country,state,phone
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4,34.0469,-118.257,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5044,4.5,34.0615,-118.24,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8057,4,34.0456,-118.236,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8124,4,34.0501,-118.24,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4074,4.5,34.1238,-118.269,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839


## Second loop: Grabbing results after result # 50

In [ ]:
# responseDict[response_url] = response.text
#Second loop: making multiple calls simultanuslly to Yelp's API

i = 0
j = 0

try:
    for i in range(len(temp_urls)):
        temp_response = requests.get(url = temp_urls[i], headers = HEADERS)
        
        print("=======================================================")
        print(f"temp_response: {temp_response}")
        
        temp_response_json = temp_response.json()
        
        temp_response_string = temp_response.text
        
        responseDict[temp_response.url] = temp_response_string
        

        print("=======================================================")
        print(f"OUTTER LOOP BEGGINS: URL: {temp_response.url}" )

        for j in range(len(temp_response_json['businesses'])):
            print("=======================================================")
            detail = temp_response_json['businesses'][j]
            print(f"inner loop (building df): biz index: {index_}: {temp_response_json['businesses'][j]['name']}")
                  
            biz_df.set_value(index_, "id", json_response['businesses'][j]['id'])
            biz_df.set_value(index_, "alias", json_response['businesses'][j]['alias'])

            biz_df.set_value(index_, "name", json_response['businesses'][j]['name'])
            biz_df.set_value(index_, "categories", ', '.join([val['title'] for val in json_response['businesses'][j]['categories']]) )        

            biz_df.set_value(index_, "review_count", json_response['businesses'][j]['review_count'])
            biz_df.set_value(index_, "rating", json_response['businesses'][j]['rating'])

            biz_df.set_value(index_, "lat", json_response['businesses'][j]['coordinates']['latitude'])
            biz_df.set_value(index_, "long", json_response['businesses'][j]['coordinates']['longitude'])

            biz_df.set_value(index_, "address", ' '.join(json_response['businesses'][j]['location']['display_address']))
            biz_df.set_value(index_, "city", json_response['businesses'][j]['location']['city'])

            biz_df.set_value(index_, "zip_code", json_response['businesses'][j]['location']['zip_code'])
            biz_df.set_value(index_, "country", json_response['businesses'][j]['location']['country'])

            biz_df.set_value(index_, "state", json_response['businesses'][j]['location']['state'])
            biz_df.set_value(index_, "phone", json_response['businesses'][j]['phone'])
            

            index_ += 1
              
        time.sleep(3)

except:
    print("Something went WRONG 😰")

print(f"Database ready: records entered:{index_}")

In [58]:
biz_df.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,zip_code,country,state,phone
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4,34.0469,-118.257,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5044,4.5,34.0615,-118.24,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8057,4,34.0456,-118.236,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8124,4,34.0501,-118.24,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4074,4.5,34.1238,-118.269,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839


In [60]:
# check the amount of results loaded into dataframe
biz_df.tail()

,id,alias,name,categories,review_count,rating,lat,long,address,city,zip_code,country,state,phone
995,ohosmz6FXVAeoW5nUkYwng,pine-and-crane-los-angeles,Pine & Crane,"Taiwanese, Chinese, Bubble Tea",2014,4.5,34.0906,-118.277,"1521 Griffith Park Blvd Los Angeles, CA 90026",Los Angeles,90026,US,CA,+13236681128
996,H9ZlIqEp930ZRniH6VRp8w,papa-cristos-los-angeles-2,Papa Cristo's,"Greek, Mediterranean",1816,4.5,34.0474,-118.3,"2771 W Pico Blvd Los Angeles, CA 90006",Los Angeles,90006,US,CA,+13237372970
997,1XUzOcwNTfky9hPAhbTkoA,los-angeles-county-museum-of-art-los-angeles-11,Los Angeles County Museum of Art,Art Museums,1791,4.5,34.0639,-118.359,"5905 Wilshire Blvd Los Angeles, CA 90036",Los Angeles,90036,US,CA,+13238576000
998,YA3bV7kd3RpWPvrarIgpWQ,milk-jar-cookies-los-angeles,Milk Jar Cookies,"Desserts, Bakeries, Coffee & Tea",1443,4.5,34.0621,-118.348,"5466 Wilshire Blvd Los Angeles, CA 90036",Los Angeles,90036,US,CA,+13236349800
999,sYn3SNQP-j2t2XSwjlCbRg,montys-good-burger-los-angeles,Monty's Good Burger,"Burgers, Vegan",1201,4.5,34.0647,-118.309,"516 S Western Ave Los Angeles, CA 90020",Los Angeles,90020,US,CA,+12139150257


# L:  load
-----------------------------------------------
* Save df to csv format

In [61]:
biz_df.to_csv(f"Resources/la_yelp_{biz_df['id'].count()}.csv", index=False)

* load dataframe into mongo

In [63]:
import pymongo

In [64]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# Select database and collection to use
db = client.la_yelp_businesses
biz_collection = db.biz_profile

In [65]:
dictData = biz_df.to_dict(orient='records')

In [66]:
biz_collection.insert_many(dictData)

## test query database

In [67]:
results = biz_collection.find()

In [68]:
type(results)

pymongo.cursor.Cursor

In [69]:
for result in results[0:2]:
    print()
    print(result)



{'_id': ObjectId('5ce068c3a3a7e537d4b0e30a'), 'id': 'TkFEKhsCixPWlShULKvMdQ', 'alias': 'bottega-louie-los-angeles', 'name': 'Bottega Louie', 'categories': 'Italian, Bakeries, Breakfast & Brunch', 'review_count': 16243, 'rating': 4.0, 'lat': 34.0469300995766, 'long': -118.256601457672, 'address': '700 S Grand Ave Los Angeles, CA 90017', 'city': 'Los Angeles', 'zip_code': '90017', 'country': 'US', 'state': 'CA', 'phone': '+12138021470'}

{'_id': ObjectId('5ce068c3a3a7e537d4b0e30b'), 'id': '7O1ORGY36A-2aIENyaJWPg', 'alias': 'howlin-rays-los-angeles-3', 'name': "Howlin' Ray's", 'categories': 'Southern, Chicken Shop, American (Traditional)', 'review_count': 5044, 'rating': 4.5, 'lat': 34.0614861063899, 'long': -118.239554800093, 'address': '727 N Broadway Ste 128 Los Angeles, CA 90012', 'city': 'Los Angeles', 'zip_code': '90012', 'country': 'US', 'state': 'CA', 'phone': '+12139358399'}


* First document in the collection

In [71]:
biz_collection.find()[0]

{'_id': ObjectId('5ce068c3a3a7e537d4b0e30a'),
 'id': 'TkFEKhsCixPWlShULKvMdQ',
 'alias': 'bottega-louie-los-angeles',
 'name': 'Bottega Louie',
 'categories': 'Italian, Bakeries, Breakfast & Brunch',
 'review_count': 16243,
 'rating': 4.0,
 'lat': 34.0469300995766,
 'long': -118.256601457672,
 'address': '700 S Grand Ave Los Angeles, CA 90017',
 'city': 'Los Angeles',
 'zip_code': '90017',
 'country': 'US',
 'state': 'CA',
 'phone': '+12138021470'}

# Create SQL database from  Dataframe

In [72]:
from sqlalchemy import create_engine

In [76]:
database_path = "Resources/la_biz.sqlite"

In [77]:
engine = create_engine(f"sqlite:///{database_path}")

In [78]:
engine.table_names()

[]

In [79]:
biz_df.to_sql(name='biz_profile', con=engine, if_exists='append', index=False)

In [80]:
engine.table_names()

['biz_profile']

### Test Query the biz_profile table


In [81]:
# pd.read_sql_query('select * from customer_name', con=engine).head()
pd.read_sql_query('select * from biz_profile', con=engine).head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,zip_code,country,state,phone
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.046930,-118.256601,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5044,4.5,34.061486,-118.239555,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8057,4.0,34.045605,-118.236061,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8124,4.0,34.050081,-118.240180,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4074,4.5,34.123840,-118.268680,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839
